In [1]:
import pymysql
wikiconn  = pymysql.connect(host='localhost',
                            user='u15068',                 # get from replica.my.cnf
                            password='G5V2xrWbdFTSkyoU',
                            port=4711,
                            db='enwiki_p',
                            charset='utf8')
wikiCur = wikiconn.cursor()

In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [3]:
import datetime
import pandas as pd

def PrintableTimestamp():
    now = datetime.datetime.now().time()
    return "[{:02d}:{:02d}:{:02d}]".format(now.hour,now.minute,now.second)

def BatchQueryFromList(listOfIDs,columnThatContainsIDs,tableToQuery,listOfColumnsToRetrieve):
    '''listOfIDs is a list of keys to retrieve from an SQL table
    columnThatContainsIDs is the column in that table that stores the keys
    tableToQuery is the table containing the information you want
    listOfColumnsToRetrieve is a list containing the SQL DB table column names you want to retrieve
    Returns a pandas dataframe with column names equivalent to those in the DB you queried'''
    baseQ = "SELECT {} FROM {} WHERE {} IN ({})"
    colsForQ = ",".join(listOfColumnsToRetrieve)
    idChunks = chunks(listOfIDs, 1000)
    result_df = pd.DataFrame()
    for ix, chunk in enumerate(idChunks):
        print(PrintableTimestamp() + " fetching chunk {}, first element {}, last element {}".format(ix,chunk[0],chunk[-1]))
        idsInChunk = ",".join(map(str,chunk))
        preppedQ = baseQ.format(colsForQ,tableToQuery,columnThatContainsIDs,idsInChunk)
        print(preppedQ)
        wikiCur.execute(preppedQ)
        if not result_df.empty:
            result_df.append(list(wikiCur.fetchall()))
        else:
            result_df = pd.DataFrame(list(wikiCur.fetchall()),columns=listOfColumnsToRetrieve)
    return result_df


In [7]:
# get IDS for EPAs from github file. For NEPAs: ../Sampling/071816_batch1_sampled_ids.csv
ids2Q = pd.read_csv("../WikiSQL/070916_edit_protected_articles.csv")  # For EPAs: ../WikiSQL/070916_edit_protected_articles.csv
ids = list(map(str, ids2Q["page_id"].values)) 
print(len(ids))

7526


In [8]:
range(1,12001)
df = BatchQueryFromList(ids,'page_id','page',['page_id','page_title'])

[09:07:50] fetching chunk 0, first element 25, last element 39942
SELECT page_id,page_title FROM page WHERE page_id IN (25,290,307,308,594,620,624,628,655,662,664,689,691,736,737,740,752,765,783,800,809,841,848,851,852,856,863,864,874,890,902,929,1014,1038,1078,1174,1178,1273,1520,1537,1560,1623,1770,1784,1806,1828,1844,1921,1973,2006,2062,2077,2129,2144,2154,2174,2176,2178,2208,2216,2273,2321,2335,2358,2369,2441,2460,2482,2500,2504,2594,2722,2905,2909,2927,2965,3201,3212,3263,3356,3363,3382,3383,3390,3410,3415,3454,3463,3466,3717,3722,3747,3778,3850,3921,3928,3956,3970,3978,3979,3986,4009,4068,4116,4173,4230,4246,4248,4294,4314,4335,4339,4375,4399,4400,4401,4431,4489,4497,4501,4531,4543,4572,4650,4654,4721,4726,4745,4849,4854,4924,4925,5052,5134,5142,5180,5211,5222,5233,5253,5278,5299,5363,5374,5382,5405,5407,5489,5551,5573,5593,5635,5655,5658,5659,5667,5668,5760,5785,5813,5820,5823,5884,5902,5910,5921,5928,5985,5987,5995,6015,6046,6088,6112,6207,6237,6271,6299,6546,6548,6598,6611,665

In [9]:
df

,page_id,page_title
0,25,b'Autism'
1,290,b'A'
2,307,b'Abraham_Lincoln'
3,308,b'Aristotle'
4,594,b'Apollo'
5,620,b'Animal_Farm'
6,624,b'Alaska'
7,628,b'Aldous_Huxley'
8,655,b'Abacus'
9,662,b'Apollo_11'
